# Testing plots of mu-lambda scatter 

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import numpy.ma as ma
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import matplotlib.dates as dates
from mpl_toolkits.axes_grid1 import ImageGrid,make_axes_locatable,host_subplot
# from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta
import sys
import os
import pyPIPS.utils as utils
import pyPIPS.thermolib as thermo
import pyPIPS.DSDlib as dsd
#import pyPIPS.disdrometer_module as dis
import pyPIPS.plotmodule as PIPSplot
# import pyPIPS.simulator as sim
import pyPIPS.pips_io as pipsio
import pyPIPS.PIPS as pips
import pyPIPS.parsivel_params as pp
import pyPIPS.parsivel_qc as pqc
# from pyCRMtools.modules import plotmodule as plotmod
from pyCRMtools.modules import utils as CRMutils
# from pyCRMtools.pycaps import arps_read
# from pyCRMtools.pycaps import pycaps_fields
# from pyCRMtools.pycaps import calvars_radar as radar
import pandas as pd
import xarray as xr
import glob
import numpy.random as random
from scipy.stats import gamma, uniform
from scipy.special import gamma as gammafunc
from scipy import ndimage
from metpy.plots import StationPlot
from metpy.calc import wind_components
from metpy.cbook import get_test_data
from metpy.plots import StationPlot
from metpy.plots.wx_symbols import current_weather, sky_cover
from metpy.units import units
%matplotlib inline

In [ ]:
# Read in one of the PIPS datasets which has the CG coefficients as attributes
PIPS_dir = '/Volumes/scr_fast/Projects/VORTEXSE/obsdata/full_PIPS_dataset_RB15/'
# PIPS_dir = '/Users/dawson29/sshfs_mounts/depot/data/Projects/VORTEXSE/obsdata/full_PIPS_dataset_RB15/'
# PIPS_filename = 'ND_avg_full_dataset_qc_60s.nc'
PIPS_filename = 'parsivel_combined_FMCW_2017_043017_PIPS2A_60s.nc'
PIPS_path = os.path.join(PIPS_dir, PIPS_filename)
PIPS_ds_read = xr.load_dataset(PIPS_path)

# Or read in full dataset for sigma-Dm plot
# PIPS_filenames = glob.glob(PIPS_dir + '/parsivel_combined*60s*nc')
# PIPS_ds_read = xr.open_mfdataset(PIPS_filenames, combine='nested', concat_dim="time", 
#                                  data_vars='minimal', coords='minimal', compat='override', parallel=True)
# Filter on RR > 0.1 mm/h
# PIPS_ds = PIPS_ds_read.where(PIPS_ds_read['rainrate_derived_qc'] > 0.1, drop=True)
PIPS_ds = PIPS_ds_read

In [ ]:
print(PIPS_ds)
PIPS_ds = PIPS_ds.set_coords(['diameter', 'min_diameter', 'max_diameter'])
print(PIPS_ds)
ND = PIPS_ds['SATP_ND_full_dataset_qc']
print(ND)
# ND = ND.assign_coords(diameter=ND.diameter_bin)
# print(ND)

In [ ]:
PIPS_ds = PIPS_ds_read.where(PIPS_ds_read['rainrate_derived_qc'] > 0.1, drop=True)

In [ ]:
DSD_MM_fit = PIPS_ds['DSD_MM234_qc']
print(DSD_MM_fit)
# Drop points where mu > 30 or lambda > 20000
DSD_MM_fit = DSD_MM_fit.where(DSD_MM_fit.sel(parameter='lamda') < 20000.)
DSD_MM_fit = DSD_MM_fit.where(DSD_MM_fit.sel(parameter='alpha') < 30.)
DSD_MM_fit = DSD_MM_fit.dropna(dim='time', how='any')
print(DSD_MM_fit)
lamda = DSD_MM_fit.sel(parameter='lamda')
mu = DSD_MM_fit.sel(parameter='alpha')



fig, ax = plt.subplots(figsize=(10, 10))
ax.scatter(lamda, mu)
print(len(lamda))

In [ ]:
# This is a handy way to deal with the problem of duplicate times that you still want to keep by
# resetting the time index and making the corresponding time coordinate contain the integer position
# of each time. Then, below, the .sel operation to extract a given time will work even if it is a duplicate
lamda_reset = lamda.reset_index("time")
lamda_reset.coords["tindex"] = lamda_reset.get_index("time")
lamda_good_vals = np.isfinite(lamda_reset)
print(lamda_good_vals)
#print(lamda_good_vals.get_index("time"))
bad_times = lamda_good_vals.where(~lamda_good_vals, drop=True)
print(bad_times)
bad_times = bad_times["tindex"]
print(bad_times)

lamda_good_vals2 = lamda_reset.where(lamda_reset > 0.)
lamda_bad_vals2 = lamda_reset.where(lamda_reset <= 0., drop=True)
bad_times2 = lamda_bad_vals2["tindex"]
print(bad_times2)
lamda_bad = lamda_reset.sel(time=bad_times).values
print(lamda_bad)
# print(lamda_good_vals.where(~lamda_good_vals, drop=True))
mu_reset = mu.reset_index('time')
mu_bad = mu_reset.sel(time=bad_times).values
print(mu_bad)


# MM_poly_coeff, MM_poly = dsd.calc_CG_polynomial(lamda, mu)